In [2]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QComboBox, QLineEdit, QLabel, QInputDialog, QScrollArea, QErrorMessage, QSlider
from PyQt5.QtCore import Qt
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure
import numpy as np
import random

app = QApplication([])
window = QWidget()

layout = QVBoxLayout()
input_layout = QVBoxLayout()

degree_label = QLabel("Select degree:")

combo = QComboBox()
combo.addItems(["1 (Linear)", "2 (Quadratic)", "3 (Cubic)", "4 (Quartic)", "n-th degree"])

generate_button = QPushButton("Genereate random values")
clear_button = QPushButton("Clear input fields")
generate_plot_button = QPushButton("Plot")

wid = QWidget()
scroll_area = QScrollArea()
scroll_area.setWidget(wid)
scroll_area.setWidgetResizable(True)
scroll_area.setFixedHeight(300)

fig = Figure (figsize=(5,4), dpi = 100)
canvas = FigureCanvasQTAgg(fig)
axes = fig.add_subplot(111)

slider = QSlider(Qt.Horizontal)
slider.setMinimum(0)
slider.setMaximum(100)
slider.setValue(50)

slider_value_label = QLabel(f"Current t value:{slider.value()/100}")

index = combo.currentIndex()+2
previous_index = combo.currentIndex()
control_points = []

for ind in range(index):
    row = QWidget()
    layout_row = QHBoxLayout()
    layout_row.setContentsMargins(0, 0, 0, 0)
    row.setLayout(layout_row)
    label = QLabel(f'P{ind+1}')
    layout_row.addWidget(label)
        
    line_edit1 = QLineEdit()
    line_edit1.setPlaceholderText('Set x')
    line_edit1.setFixedSize(100, 20)
    layout_row.addWidget(line_edit1)
        
    line_edit2 = QLineEdit()
    line_edit2.setPlaceholderText('Set y')
    line_edit2.setFixedSize(100, 20)
    layout_row.addWidget(line_edit2)
        
    input_layout.addWidget(row)

def modifyGUI():
    global previous_index

    count = 0
    i = input_layout.count() - 1
    
    if(combo.currentIndex() != 4):
        while i >= 0:
            item = input_layout.itemAt(i)
            if item.widget() is not None:
                item.widget().deleteLater()
                input_layout.removeWidget(item.widget())
                i -= 1

    if(combo.currentIndex() == 4):
        num, ok = QInputDialog.getInt(window, "Degree input dialog", "Enter a degree",value =5, min = 5, max = 100)
        if ok:
            count = num
            print(count)
            while i >= 0:
                item = input_layout.itemAt(i)
                if item.widget() is not None:
                    item.widget().deleteLater()
                    input_layout.removeWidget(item.widget())
                    i -= 1
        else:
            print(previous_index)
            combo.setCurrentIndex(previous_index)
            return
    else:
        count = combo.currentIndex() + 1
   
    for j in range(count+1):
        row = QWidget()
        layout_row = QHBoxLayout()
        layout_row.setContentsMargins(0, 0, 0, 0)
        row.setLayout(layout_row)
        label = QLabel(f'P{j+1}')
        layout_row.addWidget(label)
        
        line_edit1 = QLineEdit()
        line_edit1.setPlaceholderText('Set x')
        line_edit1.setFixedSize(100, 20)
        layout_row.addWidget(line_edit1)
        
        line_edit2 = QLineEdit()
        line_edit2.setPlaceholderText('Set y')
        line_edit2.setFixedSize(100, 20)
        layout_row.addWidget(line_edit2)
        
        row.setFixedHeight(20)
        input_layout.addWidget(row)
    
    wid.adjustSize()
    window.adjustSize()
    previous_index = combo.currentIndex()

def linear_interp(p1, p2, t):
    return (1 - t) * p1 + t * p2

def de_casteljau(control_points, t):
    points = np.array(control_points)
    n = len(points)
    intermediate_points = []
    for r in range(1, n):
        new_points = []
        for i in range(n - r):
            new_point = linear_interp(points[i], points[i + 1], t)
            new_points.append(new_point)
        points = np.array(new_points)
        intermediate_points.append(points.copy())
    return points[0], intermediate_points

def slider_value_changed():
    t = slider.value()/100
    slider_value_label.setText(f"Current t value:{t}")
    if control_points:
        update_plot(t)
    
def generate_plot():
    global control_points
    control_points = []
    t = slider.value()/100
    
    for i in range (input_layout.count()):
       
        row_widget = input_layout.itemAt(i).widget()
        row_layout = row_widget.layout()
        x_input_edit = row_layout.itemAt(1).widget()
        y_input_edit = row_layout.itemAt(2).widget()
        x_input = x_input_edit.text()
        y_input = y_input_edit.text()
        
        try:
            x = float(x_input)
            y = float(y_input)
            point = [x,y]
            control_points.append(point)
        except ValueError:
            err = QErrorMessage()
            err.showMessage("Invalid input. Please input numerical values.")
            err.exec_()
            return
        if len(control_points) == input_layout.count():
            print(len(control_points), input_layout.count())
            update_plot(t)


def update_plot(t):
    global control_points
    axes.clear()
    bezier_curve_points_list = []
    id = 1
    control_points_arr = np.array(control_points)
    axes.plot(control_points_arr[:,0], control_points_arr[:,1], 'ro-', label = 'Control points')
    final, points = de_casteljau(control_points_arr, t)
    for q in range (101):
        point = de_casteljau(control_points_arr, q/100)[0]
        bezier_curve_points_list.append(point)
    bezier_curve = np.array(bezier_curve_points_list)
    for intermediate in points:
        axes.plot(intermediate[:,0], intermediate[:,1], marker = 'o', linestyle = '-', label = f'Segment{id}')
        id += 1
    axes.plot(bezier_curve[:, 0], bezier_curve[:, 1], linestyle = '--', color = 'black', label = 'Bezier curve')
    axes.legend()
    canvas.draw()     

def generate_values():
    for i in range(input_layout.count()):
        row_widget = input_layout.itemAt(i).widget()
        row_layout = row_widget.layout()
        row_layout.itemAt(1).widget().setText(str(random.randrange(-200,200)))
        row_layout.itemAt(2).widget().setText(str(random.randrange(-200,200)))   

def clear_fields():
    for i in range(input_layout.count()):
        row_widget = input_layout.itemAt(i).widget()
        row_layout = row_widget.layout()
        row_layout.itemAt(1).widget().setText("")
        row_layout.itemAt(2).widget().setText("")   

"""combo.currentIndexChanged().connect(modifyGUI)"""
combo.activated.connect(modifyGUI)
slider.valueChanged.connect(slider_value_changed)
generate_button.clicked.connect(generate_values)
clear_button.clicked.connect(clear_fields)
generate_plot_button.clicked.connect(generate_plot)

layout.addWidget(degree_label)
layout.addWidget(combo)
layout.addWidget(scroll_area)
layout.addWidget(clear_button)
layout.addWidget(generate_button)
layout.addWidget(generate_plot_button)
layout.addWidget(canvas)
layout.addWidget(slider)
layout.addWidget(slider_value_label)

wid.setLayout(input_layout)
window.setLayout(layout)
window.show()
app.exec()


2 2
2 2
2 2
2 2
2 2


0

In [1]:
import sys
import numpy as np
from PyQt5.QtWidgets import (QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QComboBox, QLineEdit, QLabel, QInputDialog, QScrollArea, QErrorMessage, QSlider)
from PyQt5.QtCore import Qt
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure
from matplotlib.animation import FuncAnimation

app = QApplication([])
window = QWidget()
layout = QVBoxLayout()

input_layout = QVBoxLayout()

degree_label = QLabel("Select degree:")
combo = QComboBox()
combo.addItems(["1 (Linear)", "2 (Quadratic)", "3 (Cubic)", "4 (Quartic)", "n-th degree"])

index = combo.currentIndex() + 2
for ind in range(index):
    row = QWidget()
    layout_row = QHBoxLayout()
    layout_row.setContentsMargins(0, 0, 0, 0)
    row.setLayout(layout_row)
    label = QLabel(f'P{ind + 1}')
    layout_row.addWidget(label)
        
    line_edit1 = QLineEdit()
    line_edit1.setPlaceholderText('Set x')
    line_edit1.setFixedSize(100, 20)
    layout_row.addWidget(line_edit1)
        
    line_edit2 = QLineEdit()
    line_edit2.setPlaceholderText('Set y')
    line_edit2.setFixedSize(100, 20)
    layout_row.addWidget(line_edit2)
        
    input_layout.addWidget(row)

wid = QWidget()
scroll_area = QScrollArea()
scroll_area.setWidget(wid)
scroll_area.setWidgetResizable(True)
scroll_area.setFixedHeight(300)
layout.addWidget(degree_label)
layout.addWidget(combo)
layout.addWidget(scroll_area)

previous_index = combo.currentIndex()

def modifyGUI():
    global previous_index

    count = 0
    i = input_layout.count() - 1
    
    if combo.currentIndex() != 4:
        while i >= 0:
            item = input_layout.itemAt(i)
            if item.widget() is not None:
                item.widget().deleteLater()
                input_layout.removeWidget(item.widget())
                i -= 1

    if combo.currentIndex() == 4:
        num, ok = QInputDialog.getInt(window, "Degree input dialog", "Enter a degree", value=5, min=5, max=100)
        if ok:
            count = num
            while i >= 0:
                item = input_layout.itemAt(i)
                if item.widget() is not None:
                    item.widget().deleteLater()
                    input_layout.removeWidget(item.widget())
                    i -= 1
        else:
            combo.setCurrentIndex(previous_index)
            return
    else:
        count = combo.currentIndex() + 1
   
    for j in range(count + 1):
        row = QWidget()
        layout_row = QHBoxLayout()
        layout_row.setContentsMargins(0, 0, 0, 0)
        row.setLayout(layout_row)
        label = QLabel(f'P{j + 1}')
        layout_row.addWidget(label)
        
        line_edit1 = QLineEdit()
        line_edit1.setPlaceholderText('Set x')
        line_edit1.setFixedSize(100, 20)
        layout_row.addWidget(line_edit1)
        
        line_edit2 = QLineEdit()
        line_edit2.setPlaceholderText('Set y')
        line_edit2.setFixedSize(100, 20)
        layout_row.addWidget(line_edit2)
        
        row.setFixedHeight(20)
        input_layout.addWidget(row)
    
    wid.adjustSize()
    window.adjustSize()
    previous_index = combo.currentIndex()

combo.currentIndexChanged.connect(modifyGUI)

def linear_interp(p1, p2, t):
    return (1 - t) * p1 + t * p2

def de_casteljau(control_points, t):
    points = np.array(control_points)
    n = len(points)
    intermediate_points = []
    for r in range(1, n):
        new_points = []
        for i in range(n - r):
            new_point = linear_interp(points[i], points[i + 1], t)
            new_points.append(new_point)
        points = np.array(new_points)
        intermediate_points.append(points.copy())
    return points[0], intermediate_points

fig = Figure(figsize=(5, 4), dpi=100)
canvas = FigureCanvasQTAgg(fig)
axes = fig.add_subplot(111)

slider = QSlider(Qt.Horizontal)
slider.setMinimum(0)
slider.setMaximum(100)
slider.setValue(50)

slider_value_label = QLabel(f"Current t value: {slider.value() / 100}")

control_points = []

def slider_value_changed():
    t = slider.value() / 100
    slider_value_label.setText(f"Current t value: {t}")
    if control_points:
        update_plot(t)
    
slider.valueChanged.connect(slider_value_changed)

def generate_plot():
    global control_points
    control_points = []
    t = slider.value() / 100
    
    for i in range(input_layout.count()):
        row_widget = input_layout.itemAt(i).widget()
        row_layout = row_widget.layout()
        x_input_edit = row_layout.itemAt(1).widget()
        y_input_edit = row_layout.itemAt(2).widget()
        x_input = x_input_edit.text()
        y_input = y_input_edit.text()
        
        try:
            x = float(x_input)
            y = float(y_input)
            point = [x, y]
            control_points.append(point)
        except ValueError:
            err = QErrorMessage()
            err.showMessage("Invalid input. Please input numerical values.")
            err.exec_()
            return
    
    if control_points:
        update_plot(t)

def update_plot(t):
    global control_points
    axes.clear()
    bezier_curve_points_list = []
    id = 1
    control_points_arr = np.array(control_points)
    axes.plot(control_points_arr[:, 0], control_points_arr[:, 1], 'ro-', label='Control points')
    final, points = de_casteljau(control_points_arr, t)
    for q in range(101):
        point = de_casteljau(control_points_arr, q / 100)[0]
        bezier_curve_points_list.append(point)
    bezier_curve = np.array(bezier_curve_points_list)
    for intermediate in points:
        axes.plot(intermediate[:, 0], intermediate[:, 1], marker='o', linestyle='-', label=f'Segment{id}')
        id += 1
    axes.plot(bezier_curve[:, 0], bezier_curve[:, 1], linestyle='--', color='black', label='Bezier curve')
    axes.legend()
    canvas.draw()

generate_plot_button = QPushButton("Plot")
generate_plot_button.clicked.connect(generate_plot)
layout.addWidget(generate_plot_button)

# Add the animation button
animate_button = QPushButton("Animate")
layout.addWidget(animate_button)

layout.addWidget(canvas)
layout.addWidget(slider)
layout.addWidget(slider_value_label)

wid.setLayout(input_layout)
window.setLayout(layout)

def animate(frame):
    t = frame / 100.0
    slider_value_label.setText(f"Current t value: {t}")
    if control_points:
        update_plot(t)

def start_animation():
    anim = FuncAnimation(fig, animate, frames=range(101), interval=50, repeat=True)
    canvas.draw()

animate_button.clicked.connect(start_animation)

window.show()
app.exec()


0

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QComboBox, QLineEdit, QLabel, QInputDialog, QScrollArea, QErrorMessage, QSlider
from PyQt5.QtCore import Qt
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure
import numpy as np
import random

class BezierApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.layout = QVBoxLayout()
        self.input_layout = QVBoxLayout()

        self.degree_label = QLabel("Select degree:")
        self.combo = QComboBox()
        self.combo.addItems(["1 (Linear)", "2 (Quadratic)", "3 (Cubic)", "4 (Quartic)", "n-th degree"])

        self.generate_button = QPushButton("Generate random values")
        self.clear_button = QPushButton("Clear input fields")
        self.generate_plot_button = QPushButton("Plot")

        self.wid = QWidget()
        self.scroll_area = QScrollArea()
        self.scroll_area.setWidget(self.wid)
        self.scroll_area.setWidgetResizable(True)
        self.scroll_area.setFixedHeight(300)

        self.fig = Figure(figsize=(5, 4), dpi=100)
        self.canvas = FigureCanvasQTAgg(self.fig)
        self.axes = self.fig.add_subplot(111)

        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.setMaximum(100)
        self.slider.setValue(50)

        self.slider_value_label = QLabel(f"Current t value: {self.slider.value() / 100}")

        self.index = self.combo.currentIndex() + 2
        self.previous_index = self.combo.currentIndex()
        self.control_points = []

        self.setup_initial_input_fields()

        self.combo.activated.connect(self.modifyGUI)
        self.slider.valueChanged.connect(self.slider_value_changed)
        self.generate_button.clicked.connect(self.generate_values)
        self.clear_button.clicked.connect(self.clear_fields)
        self.generate_plot_button.clicked.connect(self.generate_plot)

        self.layout.addWidget(self.degree_label)
        self.layout.addWidget(self.combo)
        self.layout.addWidget(self.scroll_area)
        self.layout.addWidget(self.clear_button)
        self.layout.addWidget(self.generate_button)
        self.layout.addWidget(self.generate_plot_button)
        self.layout.addWidget(self.canvas)
        self.layout.addWidget(self.slider)
        self.layout.addWidget(self.slider_value_label)

        self.wid.setLayout(self.input_layout)
        self.setLayout(self.layout)
        self.show()

    def linear_interp(self, p1, p2, t):
        return (1 - t) * p1 + t * p2

    def de_casteljau(self, control_points, t):
        points = np.array(control_points)
        n = len(points)
        intermediate_points = []
        for r in range(1, n):
            new_points = []
            for i in range(n - r):
                new_point = self.linear_interp(points[i], points[i + 1], t)
                new_points.append(new_point)
            points = np.array(new_points)
            intermediate_points.append(points.copy())
        return points[0], intermediate_points
    
    def setup_initial_input_fields(self):
        for ind in range(self.index):
            row = QWidget()
            layout_row = QHBoxLayout()
            layout_row.setContentsMargins(0, 0, 0, 0)
            row.setLayout(layout_row)
            label = QLabel(f'P{ind + 1}')
            layout_row.addWidget(label)

            line_edit1 = QLineEdit()
            line_edit1.setPlaceholderText('Set x')
            line_edit1.setFixedSize(100, 20)
            layout_row.addWidget(line_edit1)

            line_edit2 = QLineEdit()
            line_edit2.setPlaceholderText('Set y')
            line_edit2.setFixedSize(100, 20)
            layout_row.addWidget(line_edit2)

            self.input_layout.addWidget(row)

    def modifyGUI(self):
        count = 0
        i = self.input_layout.count() - 1

        if self.combo.currentIndex() != 4:
            while i >= 0:
                item = self.input_layout.itemAt(i)
                if item.widget() is not None:
                    item.widget().deleteLater()
                    self.input_layout.removeWidget(item.widget())
                    i -= 1

        if self.combo.currentIndex() == 4:
            num, ok = QInputDialog.getInt(self, "Degree input dialog", "Enter a degree", value=5, min=5, max=100)
            if ok:
                count = num
                while i >= 0:
                    item = self.input_layout.itemAt(i)
                    if item.widget() is not None:
                        item.widget().deleteLater()
                        self.input_layout.removeWidget(item.widget())
                        i -= 1
            else:
                self.combo.setCurrentIndex(self.previous_index)
                return
        else:
            count = self.combo.currentIndex() + 1

        for j in range(count + 1):
            row = QWidget()
            layout_row = QHBoxLayout()
            layout_row.setContentsMargins(0, 0, 0, 0)
            row.setLayout(layout_row)
            label = QLabel(f'P{j + 1}')
            layout_row.addWidget(label)

            line_edit1 = QLineEdit()
            line_edit1.setPlaceholderText('Set x')
            line_edit1.setFixedSize(100, 20)
            layout_row.addWidget(line_edit1)

            line_edit2 = QLineEdit()
            line_edit2.setPlaceholderText('Set y')
            line_edit2.setFixedSize(100, 20)
            layout_row.addWidget(line_edit2)

            row.setFixedHeight(20)
            self.input_layout.addWidget(row)

        """self.axes.clear()
        self.canvas.draw()"""
        self.wid.adjustSize()
        self.adjustSize()
        self.previous_index = self.combo.currentIndex()

    def slider_value_changed(self):
        t = self.slider.value() / 100
        self.slider_value_label.setText(f"Current t value: {t}")
        if self.control_points:
            self.update_plot(t)

    def generate_plot(self):
        self.control_points = []
        t = self.slider.value() / 100

        for i in range(self.input_layout.count()):
            row_widget = self.input_layout.itemAt(i).widget()
            row_layout = row_widget.layout()
            x_input_edit = row_layout.itemAt(1).widget()
            y_input_edit = row_layout.itemAt(2).widget()
            x_input = x_input_edit.text()
            y_input = y_input_edit.text()

            try:
                x = float(x_input)
                y = float(y_input)
                point = [x, y]
                self.control_points.append(point)
            except ValueError:
                self.show_error_message("Invalid input. Please input numerical values.")
                return

        if len(self.control_points) == self.input_layout.count():
            self.update_plot(t)

    def update_plot(self, t):
        self.axes.clear()
        bezier_curve_points_list = []
        id = 1
        control_points_arr = np.array(self.control_points)
        self.axes.plot(control_points_arr[:, 0], control_points_arr[:, 1], 'ro-', label='Control points')
        _, points = self.de_casteljau(control_points_arr, t)
        for q in range(101):
            point = self.de_casteljau(control_points_arr, q / 100)[0]
            bezier_curve_points_list.append(point)
        bezier_curve = np.array(bezier_curve_points_list)
        for intermediate in points:
            self.axes.plot(intermediate[:, 0], intermediate[:, 1], marker='o', linestyle='-', label=f'Segment{id}')
            id += 1
        self.axes.plot(bezier_curve[:, 0], bezier_curve[:, 1], linestyle='--', color='black', label='Bezier curve')
        self.axes.legend()
        self.canvas.draw()

    def generate_values(self):
        for i in range(self.input_layout.count()):
            row_widget = self.input_layout.itemAt(i).widget()
            row_layout = row_widget.layout()
            row_layout.itemAt(1).widget().setText(str(random.randrange(-200, 200)))
            row_layout.itemAt(2).widget().setText(str(random.randrange(-200, 200)))

    def clear_fields(self):
        for i in range(self.input_layout.count()):
            row_widget = self.input_layout.itemAt(i).widget()
            row_layout = row_widget.layout()
            row_layout.itemAt(1).widget().setText("")
            row_layout.itemAt(2).widget().setText("")


if __name__ == '__main__':
    app = QApplication([])
    ex = BezierApp()
    app.exec_()
